# Introduction 

### About the competition:
The Tabular Playground Series March 2021 is relatively simpler, beginner-friendly competition which includes primary tabular data to play with and getting a good grasp on fundamentals.
</br> 
This notebook demonstrates a basic pipeline using a neural network with hyper-parameter tuning using Optuna framework. We will also taking a look at visualizaion and some EDA. Finally the notebook concludes the important learnings for the competition/training.

In [ ]:
#imports 
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import sklearn.datasets
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import optuna 
import os
#models
import lightgbm as lgb
import xgboost as xgb
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


# Plots
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#ignoring warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train_df.head()

Seperating training data with features from train_df

In [ ]:
cat_features = [f'cat{x}'for x in range(0,19)]
cont_features = [f'cont{x}'for x in range(0,11)]
features = cat_features + cont_features

data = train_df[features]
X_test = test_df[features]
target = train_df['target']

train_data = train_df[features]
train_data.head()

## Part 1 - Automated EDA

In [ ]:
#install sweetviz library for automated eda
!pip install sweetviz

In [ ]:
%%time
#analyzing data using sweetviz (automated EDA)
import sweetviz as sv
data_report = sv.analyze(train_df)
data_report.show_html('Analysis.html')


In [ ]:
from IPython.display import IFrame
IFrame(src = 'Analysis.html',width=1000,height=600)

There are a few conclusions we can make from this EDA:
* No feature contains missing/Null values
* There are 19 categorical features, 11 categorical features and 1 target which takes only two values (true/false or 0/1)
* cat2, cat3, cat4, cat5, cat6 and cat9 has small portion of distinct values(Hypothesis: - which can be encoded as "others" without affecting significant performace)
* The range of continuous variable are relative same which means we don't need to change the distribution of them.

## Part 2 - Training Data Visualization

Let's take a quick look at training data distribution and what can we learn more about them.

In [ ]:
#setting up the platform
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))

sns.countplot(target, color="g")
ax.xaxis.grid(False)
ax.set(ylabel="Values")
ax.set(xlabel="Target")
ax.set(title="Target values distribution")
sns.despine(trim=True, left=True)
plt.show()

Comparing train and test data distribution for continuous variables 

In [ ]:

# we will look into the features distribution now, to get insight into the data
plt.figure()
fig, ax = plt.subplots(4, 3,figsize=(14, 20))
for i,feature in enumerate(cont_features):
    plt.subplot(4, 3,i+1)
    sns.distplot(train_data[feature],color="blue", kde=True,bins=120, label='train')
    sns.distplot(test_df[feature],color="red", kde=True,bins=120, label='test')
    plt.xlabel(feature, fontsize=9); plt.legend()
plt.show()



It seems that the distribution for train and test set matches, meaning the data is sampled from the same distribution which is good for us.

In [ ]:
#we will take a look at the correlation of features
corr = train_data.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corr, vmax=0.9, cmap="inferno", square=True)

## Part 3 - Feature Engineering

First we will remove redundant values in categorical features by applying and filtering others function from values, then we will label encode them to use as numeric features for our training dataset.

In [ ]:
#The next portion of code is adapted from this great kernal by andreshg
#https://www.kaggle.com/andreshg/tps-march-a-complete-study

#FE for cat2 var
train_data['cat2'] = train_data['cat2'].apply(lambda x: x if x not in['A', 'C', 'D', 'G', 'F', 'J', 'I', 'M', 'Q', 'L', 'O'] else 'CAT2')
X_test['cat2'] = X_test['cat2'].apply(lambda x: x if x not in['A', 'C', 'D', 'G', 'F', 'J', 'I', 'M', 'Q', 'L', 'O'] else 'CAT2')

# Fix cat5 variable
train_data['cat5'] = train_data['cat5'].apply(lambda x: x if x not in ['N', 'BS', 'BG', 'CA', 'BK', 'T', 'BO', 'CI', 'AY', 'BV'] else 'CAT50')
X_test['cat5'] = X_test['cat5'].apply(lambda x: x if x not in ['BL', 'BJ', 'CK', 'AH', 'I', 'F', 'C', 'AQ', 'AM', 'X'] else 'CAT51')

X_test['cat5'] = X_test['cat5'].apply(lambda x: x if x not in ['N', 'BS', 'BG', 'CA', 'BK', 'T', 'BO', 'CI', 'AY', 'BV'] else 'CAT50')
X_test['cat5'] = X_test['cat5'].apply(lambda x: x if x not in ['BL', 'BJ', 'CK', 'AH', 'I', 'F', 'C', 'AQ', 'AM', 'X'] else 'CAT51')

# Fix cat7 variable
train_df['cat7'] = train_df['cat7'].apply(lambda x: x if x not in ['R', 'AA', 'AY', 'AP', 'O', 'Y'] else 'CAT70')
train_df['cat7'] = train_df['cat7'].apply(lambda x: x if x not in ['AL', 'AD', 'L', 'AC', 'V', 'BA'] else 'CAT71')

X_test['cat7'] = X_test['cat7'].apply(lambda x: x if x not in ['R', 'AA', 'AY', 'AP', 'O', 'Y'] else 'CAT70')
X_test['cat7'] = X_test['cat7'].apply(lambda x: x if x not in ['AL', 'AD', 'L', 'AC', 'V', 'BA'] else 'CAT71')

# Fix cat8 variable
train_df['cat8'] = train_df['cat8'].apply(lambda x: x if x not in ['BC', 'BJ', 'I', 'AW', 'AS'] else 'CAT80')
train_df['cat8'] = train_df['cat8'].apply(lambda x: x if x not in ['AO', 'AG', 'F', 'BD', 'A'] else 'CAT81')

X_test['cat8'] = X_test['cat8'].apply(lambda x: x if x not in ['BC', 'BJ', 'I', 'AW', 'AS'] else 'CAT80')
X_test['cat8'] = X_test['cat8'].apply(lambda x: x if x not in ['AO', 'AG', 'F', 'BD', 'A'] else 'CAT81')

# Fix cat10 variable
train_df['cat10'] = train_df['cat10'].apply(lambda x: x if x not in ['DF', 'IG', 'HJ', 'EK', 'GE', 'LN', 'HB'] else 'CAT10')
train_df['cat10'] = train_df['cat10'].apply(lambda x: x if x not in ['FR', 'JR', 'GI', 'GK', 'MC', 'HC', 'CD'] else 'CAT11')

X_test['cat10'] = X_test['cat10'].apply(lambda x: x if x not in ['DF', 'IG', 'HJ', 'EK', 'GE', 'LN', 'HB'] else 'CAT10')
X_test['cat10'] = X_test['cat10'].apply(lambda x: x if x not in ['FR', 'JR', 'GI', 'GK', 'MC', 'HC', 'CD'] else 'CAT11')

In [ ]:
#label encoding for the categorical values
def label_encoder(train_data, test_data, column):
    le = LabelEncoder()
    new_feature = f"{column}_le"
    le.fit(train_data[column].unique().tolist() + test_data[column].unique().tolist())
    
    train_data[new_feature] = le.transform(train_data[column])
    test_data[new_feature] = le.transform(test_data[column])
    
    return new_feature

In [ ]:
%%time
#encoding 
EncodedFeatures = [label_encoder(train_data, X_test, feature) for feature in cat_features]
EncodedFeatures

We will now make traing dataset from label encoded categorical features + previous continous features, then using train, validation split to train our model and finally evaluate our NN based on validation dataset

In [ ]:
Train_features = EncodedFeatures + cont_features
X_train = train_data[Train_features]
X_train.head()

Splitting train-validation data for further evaluation, we will use 5% data as validation data because our training dataset consist of 300,000 values and around 15000 would be enough to evaluate performance.The first step is to create dataset class which further called to make a dataloader(which can make better use of multiprocessing)

In [ ]:
#make a class to make dataset
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        ID = self.list_IDs[index]
        # Load data and get label
        X = X_train.loc[ID].to_numpy()
        y = self.labels[ID]

        return X, y

In [ ]:
target = train_df['target']
training_set = Dataset(X_train.index, target.values)

def get_data_generators(params):    
    # Load dataset and return train validation split.
    train_size = int(0.95 * len(training_set))
    test_size = len(training_set) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(training_set, [train_size, test_size])

    train_generator = torch.utils.data.DataLoader(train_dataset, **params)
    val_generator = torch.utils.data.DataLoader(val_dataset, **params)
    
    return train_generator, val_generator


### Part 4 - Hyper-parameter Tuning with Optuna
In this section, we will tune our hyperparameters using a optimization framework called Optuna. It works by first creating an objective function which includes defining our model, then another part creates a study where our objective model is sampled using different parameters. Interally, the parameters are first sampled using different samplers to find optimum value from previous studies, then pruners are used to efficiently find parameters that work by prunning non-optimal values. The key to the optimization is providing efficient and fast hyperparameter search with distributed processing, in this notebook we will be using gpu to make search faster for our model parameters.

In [ ]:
#define constants 
use_cuda = torch.cuda.is_available()
# DEVICE = torch.device("cuda:0" if use_cuda else "cpu")
#currently we only use CPU
DEVICE = 'cpu'
torch.backends.cudnn.benchmark = True

BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

params = {'batch_size': BATCHSIZE,
          'shuffle': True,
          'num_workers': 1}
EPOCHS = 10

#make a model Neural Network
def define_model(trial): 
    #Suggest number of layers for our model.
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = []
    #number of input neurons/features
    in_features = 30
    for i in range(n_layers):
        out_features = trial.suggest_int('n_units_l{}'.format(i), 4, 128)
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(torch.nn.ReLU())
        #set dropout for regularization
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
    
    classes = 1
    layers.append(torch.nn.Linear(in_features, classes))
    layers.append(torch.nn.Sigmoid())
    return torch.nn.Sequential(*layers)


#We will define objective function for actual training during study.
def objective(trial):
    
    # Generate the model.
    model = define_model(trial).to(DEVICE)
     # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    #Splitting into 95-5 train validation data.
    train_loader, valid_loader = get_data_generators(params)
    
    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.to(DEVICE), target.to(DEVICE)
            
            #Actual training, propogating loss using optimizer.
            optimizer.zero_grad()
            output = model(data.float())
            msloss = nn.MSELoss()
            loss = msloss(output.float(),target.float())
            loss.backward()
            optimizer.step()
    
    
        # Validation of the model.
        model.eval()
        preds = []
        y_test = []
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data.float())
                preds.extend(output.numpy())
                y_test.extend(target.numpy())
        
        preds = np.array(np.array(preds) > 0.5,dtype=int)
        # print("acc", accuracy_score(y_test,preds))
        if len(np.unique(preds)) != 2:
            auc_score = 0
        else:
            auc_score = roc_auc_score(y_test,preds)
        trial.report(auc_score, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    #calculate auc loss
    return auc_score

In [ ]:
#ignore debug code 
# Validation of the model.
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# model.eval()
# preds = []
# y_test = []
# with torch.no_grad():
#     for batch_idx, (data, target) in enumerate(valid_loader):
#         # Limiting validation data.
#         if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
#             break
#         data, target = data.to(DEVICE), target.to(DEVICE)
#         output = model(data.float())
#         preds.append(output.cpu().numpy().reshape((BATCHSIZE,)))
#         y_test.append(target.cpu().numpy())
# print("acc", accuracy_score(y_test,preds))
# auc_score = roc_auc_score(y_test,preds)
# np.array(preds.cpu().numpy(),dtype='float')
# preds = []
# y_test = []
# with torch.no_grad():
#     for batch_idx, (data, target) in enumerate(valid_loader):
#         # Limiting validation data.
#         if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
#             break
#         data, target = data.to(DEVICE), target.to(DEVICE)
#         output = model(data.float())
#         preds.extend(output.numpy())
#         y_test.extend(target.numpy())
# np.array(np.array(preds) > 0.3,dtype=int).reshape(-1,BATCHSIZE)
# for k,l in valid_loader:
#     data, target = data.to(DEVICE), target.to(DEVICE)
#     output = model(data.float())
    
#     print(output.reshape((BATCHSIZE,)).shape, target.shape)
#     loss = nn.MSELoss()
#     loss_val = loss(output.float(),target.float())
#     loss_val.backward()
#     break
# model(torch.from_numpy(X_train.loc[0].to_numpy()).float())

Creating a study for trials of hyperparameters, we are maximizing the area under ROC curve which is the evaluation measure used in this competition

In [ ]:
#Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40, timeout=1000)

In [ ]:
print(f"Value of Best trial is: {study.best_trial.value}")
print("Best value Params: ")
for key, value in study.best_trial.params.items():
    print("    {}: {}".format(key, value))

## Part 5 - Final Model and Final Predictions

We will use best trial parameters from above and train a model on full training data, which we will use for final predictions

In [ ]:
#define constants 
use_cuda = torch.cuda.is_available()
# DEVICE = torch.device("cuda:0" if use_cuda else "cpu")
DEVICE = 'cpu'
torch.backends.cudnn.benchmark = True

BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

params = {'batch_size': BATCHSIZE,
          'shuffle': True,
          'num_workers': 4}
EPOCHS = 10
layers = []
#number of input neurons/features
in_features = 30
for i in range(2):
    out_features = 50
    layers.append(torch.nn.Linear(in_features, out_features))
    layers.append(torch.nn.ReLU())
    #set dropout for regularization
    p = 0.4928102110636615
    layers.append(nn.Dropout(p))
    in_features = out_features

classes = 1
layers.append(torch.nn.Linear(in_features, classes))
layers.append(torch.nn.Sigmoid())
model = torch.nn.Sequential(*layers).to(DEVICE)
lr = 0.001879813193722479
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)
    
train_generator = torch.utils.data.DataLoader(training_set, **params)


# Training of the model.
for epoch in range(50):
    print(f"Epoch no: {epoch}")
    model.train()
    for batch_idx, (data, target) in enumerate(train_generator):
        # Limiting training data for faster epochs.
        if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
            break

        data, target = data.to(DEVICE), target.to(DEVICE)

        #Actual training, propogating loss using optimizer.
        optimizer.zero_grad()
        output = model(data.float())
        msloss = nn.MSELoss()
        loss = msloss(output.float(),target.float())
#         print(loss)
        loss.backward()
        optimizer.step()

#we won't be validating this model as we already know this is best from above and use this to train on full data

**Note**:
This is only proof of concept for finding optimized hyperparameters of a neural network, your values of best trial may differ from those of here due to inherent non-determinism. Trying for more number of trials is one of the best way to find best_trial which are optimized. 

In [ ]:
#predictions

with torch.no_grad():
    test_data = X_test[Train_features].to_numpy()
    predictions = model(torch.from_numpy(test_data).float())
    final_preds = (np.concatenate(predictions.numpy()) > 0.3).astype(int)
    sub_df['target'] = predictions.numpy()
    sub_df.to_csv('FinalSubmission.csv')

## Part 6 - Conclusion

Although Neural Networks are very powerful when it comes to retaining complexity of data and learning non-linear functions from the internal patterns of training data, it is prone to overfitting and we cann't see much deeper inside of it. It's like blackbox which is very powerful yet unknown from inside. But it doesn't prevent us to make better use of it. In this notebook we have demonstrated the concept of hyperparameter optimization using optuna which can be further used to find best model for training data given the evaluation metric. If you find this notebook interesting, don't forget to upvote and give your feedback down below. Thanks for reading and your valuable time!!